Connect to Azure Workspace and Create Environment

Connect to your Azure workspace using a configuration file
and create an environment from a Conda specification file.


In [ ]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()

env = Environment.from_conda_specification(
    name='myenv',
    file_path='environment.yaml'
)

env.register(workspace=ws)

Set Up Compute Target

Set up a compute target in Azure using an existing compute cluster.


In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_target = ComputeTarget(workspace=ws, name="pedrovillabe169")

Import Required Libraries

Import necessary libraries including TensorFlow, NumPy, 
and various utilities for processing images and data.


In [ ]:
import tensorflow as tf 
from tensorflow.keras import layers, Model
import numpy as np
import json
import matplotlib.pyplot as plt
import tifffile as tiff
import os
import pandas as pd
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import re
from datetime import datetime
from os.path import join
from collections import defaultdict
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Masking, ConvLSTM2D, BatchNormalization, Dense, Flatten, concatenate, GlobalAveragePooling2D, Embedding, TimeDistributed, GlobalAveragePooling3D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K
import tensorflow as tf
from keras.layers import Reshape
from tensorflow.keras.regularizers import l2
from sklearn.utils import class_weight
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

Load Data from JSON Files

Load Sentinel-1 and Sentinel-2 metadata from JSON files 
containing information about the satellite images.


In [ ]:
def load_json_data(json_file):
    with open(json_file, 'r') as file: 
        data = json.load(file)
    return data 

azure_base_path = 'SEN12FLOOD'

data_s1 = load_json_data(join(azure_base_path, 'S1list.json'))
data_s2 = load_json_data(join(azure_base_path, 'S2list.json'))

Define Helper Functions

Define helper functions for image processing, including combining 
Sentinel-1 polarizations and Sentinel-2 bands, and for extracting dates.


In [ ]:
import numpy as np

def use_single_channel_as_grayscale(vv, vh, use_vv=True):

    return vv if use_vv else vh

def combine_s1_polarizations(images):

    return [img['VV'] if 'VV' in img else np.zeros((522, 544)) for img in images]

def combine_s2_bands(images_dicts, desired_bands, base_resolution=10):
    combined_images = []
    for images in images_dicts:
        band_images = []
        for band in desired_bands:
            band_key = f"B{band}"
            if band_key in images:
                band_image = images[band_key]
                band_res = band_resolution(band_key)
                resolution_factor = base_resolution / band_res
                if resolution_factor != 1:
                    band_image = resize(band_image, (int(band_image.shape[0] * resolution_factor),
                                                     int(band_image.shape[1] * resolution_factor)),
                                        preserve_range=True, anti_aliasing=True)
                band_images.append(band_image)
            else:
                band_res = band_resolution(band_key)
                resolution_factor = base_resolution / band_res
                band_images.append(np.zeros((int(522 * resolution_factor), int(544 * resolution_factor))))
        
        shape = band_images[0].shape
        for i, band_image in enumerate(band_images):
            if band_image.shape != shape:
                band_images[i] = resize(band_image, shape, preserve_range=True, anti_aliasing=True)

        combined_image = np.stack(band_images, axis=-1)
        combined_images.append(combined_image)
    return combined_images

def band_resolution(band_key):

    band_resolutions = {
        "B1": 60, "B2": 10, "B3": 10, "B4": 10,
        "B5": 20, "B6": 20, "B7": 20, "B8": 10,
        "B8A": 20, "B9": 60, "B11": 20, "B12": 20
    }
    return band_resolutions.get(band_key, 10)

Load Images from Directory

Define a function to load images from a specified directory.
The function reads Sentinel-1 and Sentinel-2 images from TIFF files
and organizes them by their corresponding metadata.


In [ ]:
def load_images_from_directory(directory, s1_data, s2_data, use_vv=True, desired_bands=[2, 3, 4]):
    images = []
    filenames = []
    labels = []

    folder_name = os.path.basename(directory)
    print(f"Processing folder: {folder_name}")

    for file_name in os.listdir(directory):
        if file_name.endswith('.tif') or file_name.endswith('.tiff'):
            if file_name.startswith('S1') and file_name.endswith('_VH.tif'):
                json_data = s1_data
                layer_name = "VH"
            elif file_name.startswith('S2'):
                json_data = s2_data
                layer_name = file_name.split('_')[-1].replace('.tif', '')  # Get the last part as the layer
            else:
                print(f"Skipping file (not S1 or S2 or not ending with _VH.tif): {file_name}")
                continue

            base_filename = '_'.join(file_name.split('_')[:-1])

            found = False
            for item in json_data.get(folder_name, {}).values():
                if isinstance(item, dict) and 'filename' in item and item['filename'] in base_filename:
                    file_path = os.path.join(directory, file_name)
                    img = tiff.imread(file_path)
                    img_array = np.array(img)

                    if base_filename not in filenames:
                        filenames.append(base_filename)
                        images.append({})
                        labels.append(item.get('FLOODING', False))

                    index = filenames.index(base_filename)
                    images[index][layer_name] = img_array
                    found = True
                    break

            if not found:
                print(f"Skipping file (metadata not found): {file_name}")

    for idx, img_dict in enumerate(images):
        if any(key.startswith("B") for key in img_dict):  
            img_dict['rgb'] = combine_s2_bands([img_dict], desired_bands)[0]

    return images, filenames, labels

Extract Date from Filename

Define a function to extract dates from filenames.
This is used to sort images chronologically for each scenario.


In [ ]:
def extract_date(filename):
    match = re.search(r'\d{4}-\d{2}-\d{2}', filename)
    if match:
        return datetime.strptime(match.group(), '%Y-%m-%d').date()
    else:
        match = re.search(r'\d{8}T\d{6}', filename)
        if match:
            return datetime.strptime(match.group(), '%Y%m%dT%H%M%S').date()
    return None

Process Folders

Define a function to process all folders and load the corresponding images.
This function reads and organizes images based on the flood scenarios.


In [ ]:
def process_folders(folder_list, main_directory, s1_data, s2_data):
    scenario_data = {}
    
    for folder in folder_list:
        directory = os.path.join(main_directory, folder)
        images, filenames, labels = load_images_from_directory(directory, s1_data, s2_data)

        print(f"Folder: {folder}, Files Loaded: {len(filenames)}")  

        if not images:
            continue
        
        temp_data = []
        for img, fname, lbl in zip(images, filenames, labels):
            date = extract_date(fname)
            if date is not None:
                temp_data.append((img, fname, lbl, date))
                print(f"Filename: {fname}, Date: {date}, Label: {lbl}")  

        temp_data.sort(key=lambda x: x[3]) # ORGANIZE THE DATA BY DATE. VEY IMPORTANT FOR RESPECT TIME-SERIES
        
        scenario_data[folder] = {
            'images': [data[0] for data in temp_data],
            'filenames': [data[1] for data in temp_data],
            'labels': [data[2] for data in temp_data],
            'dates': [data[3] for data in temp_data]
        }
            
    return scenario_data



List of all the readable scenarios. About 50 scenarios were not able to proccess due to a problem with the tiff library

In [ ]:
def list_folders(main_directory):
    return [f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))]

# Main SEN12FLOOD directory, whole data is there
# main_directory = '/home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD'
main_directory = azure_base_path
# List all folders
# all_folders = list_folders(main_directory)

# Manually select folders for training and testing
a_folders = [str(i) for i in range(68)]  # Example folders for training
b_folders = ['0001', '0004', '0005', '0006', '0007', '0009', '0010', '0011', '0012', '0013', '0014', '0015', '0018', '0020', '0021', '0022', '0023', '0024', '0025', '0026', '0027', '0028', '0029', '0030', '0031', '0033', '0034', '0035', '0036', '0037', '0042', '0043', '0044', '0045', '0046', '0047', '0048', '0050', '0053', '0054', '0055', '0057', '0059', '0060', '0061', '0063', '0065', '0066', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0081', '0082', '0085', '0093', '0094', '0095', '0096', '0098', '0099', '0111', '0115', '0125', '0126', '0128', '0130', '0131', '0132', '0133', '0134', '0135', '0137', '0138', '0139', '0140', '0143', '0144', '0145', '0146', '0147', '0148', '0149', '0150', '0151', '0155', '0156', '0157', '0158', '0159', '0160', '0161', '0162', '0163', '0166', '0167', '0168', '0169', '0170', '0171', '0173', '0174', '0176', '0177', '0178', '0181', '0182', '0184', '0186', '0187', '0188', '0191', '0192', '0193', '0194', '0196', '0198', '0199', '0200', '0201', '0204', '0205', '0206', '0207', '0208', '0209', '0210', '0212', '0213', '0214', '0215', '0216', '0217', '0218', '0219', '0220', '0221', '0222', '0223', '0225', '0226', '0227', '0230', '0231', '0232', '0233', '0234', '0235', '0236', '0238', '0240', '0241', '0243', '0244', '0245', '0246', '0247', '0248', '0249', '0250', '0253', '0254', '0255', '0256', '0257', '0258', '0259', '0260', '0261', '0262', '0263', '0266', '0267', '0271', '0272', '0273', '0274', '0275', '0276', '0277', '0278', '0279', '0280', '0281', '0282', '0285', '0286', '0287', '0288', '0290', '0293', '0294', '0295', '0296', '0298', '0299', '0300', '0301', '0303', '0305', '0306', '0307', '0308', '0309', '0310', '0311', '0313', '0316', '0318', '0319', '0320', '0321', '0323', '0324', '0325', '0326', '0327', '0328', '0329', '0330', '0331', '0332', '0333', '0334', '0335', '0336']   # Example folders for testing

all_folders = a_folders + b_folders

Get Flooding and Non-Flooding Scenarios

Define a function to identify scenarios that contain flooding
and those that do not. This helps in balancing the dataset.


In [ ]:
import os

def get_flooding_and_non_flooding_scenarios(folder_list, main_directory, s1_data, s2_data):
    flooding_scenarios = []
    non_flooding_scenarios = []

    for folder in folder_list:
        scenario_data = process_folders([folder], main_directory, s1_data, s2_data)
        folder_scenario = scenario_data.get(folder)

        if folder_scenario:
            labels = folder_scenario['labels']
            if any(labels):  
                flooding_scenarios.append(folder)
            else:  
                non_flooding_scenarios.append(folder)
    
    return flooding_scenarios, non_flooding_scenarios

flooding_scenarios, non_flooding_scenarios = get_flooding_and_non_flooding_scenarios(all_folders, main_directory, data_s1, data_s2)


Select Balanced Subset of Scenarios

Define a function to select a balanced subset of flooding and non-flooding scenarios.
This is used to ensure an even distribution for training.


In [ ]:
import random

def select_balanced_subset(flooding_scenarios, non_flooding_scenarios, fraction=1):
    min_count = min(len(flooding_scenarios), len(non_flooding_scenarios))
    subset_size = int(min_count * fraction)

    flooding_subset = random.sample(flooding_scenarios, subset_size)
    non_flooding_subset = random.sample(non_flooding_scenarios, subset_size)

    return flooding_subset + non_flooding_subset

balanced_subset = select_balanced_subset(flooding_scenarios, non_flooding_scenarios, fraction=1)

Split Scenarios into Train, Validation, and Test Sets

Define a function to split the scenarios into training, validation, and testing sets.


In [ ]:
def split_scenarios(scenarios, train_ratio=0.6, val_ratio=0.2):
    random.shuffle(scenarios)
    total_size = len(scenarios)
    train_size = int(train_ratio * total_size)
    val_size = int(val_ratio * total_size)

    train_scenarios = scenarios[:train_size]
    val_scenarios = scenarios[train_size:train_size + val_size]
    test_scenarios = scenarios[train_size + val_size:]

    return train_scenarios, val_scenarios, test_scenarios



In [ ]:
train_folders, validation_folders, test_folders = split_scenarios(balanced_subset)

Process Folders for Train, Validation, and Test Sets

Process the folders for each set and store the images and metadata.


In [ ]:
train_data = process_folders(train_folders, main_directory, data_s1, data_s2)
val_data = process_folders(validation_folders, main_directory, data_s1, data_s2)
test_data = process_folders(test_folders, main_directory, data_s1, data_s2)

print(f"Training folders: {train_folders}")
print(f"Validation folders: {validation_folders}")
print(f"Testing folders: {test_folders}")

Resize and Convert Images

Define a function to resize and convert images to the target shape.
This ensures that all images have a consistent shape for model training.


In [ ]:
def resize_and_convert_images(img_dict, target_shape):
    resized_img_dict = {}
    for key, img in img_dict.items():
        if isinstance(img, np.ndarray):
            if img.ndim == 4 and img.shape[0] == 1:  # (1, H, W, C)
                img = img[0]  

            if img.ndim == 3 and img.shape[2] == 2:  # (H, W, 2)
                img = img[:, :, 0]  

            if img.ndim == 2:  
                img_resized = resize(img, target_shape[:2], preserve_range=True, anti_aliasing=True)
                img_resized = img_resized[:, :, np.newaxis]  
            elif img.ndim == 3 and img.shape[2] <= target_shape[2]:  
                img_resized = resize(img, (target_shape[0], target_shape[1], img.shape[2]), preserve_range=True, anti_aliasing=True)
            else:
                raise ValueError(f"Unexpected image shape: {img.shape}")

            resized_img_dict[key] = np.array(img_resized, dtype=np.float32)
        else:
            raise ValueError(f"Unexpected type for image data: {type(img)}")
    
    return resized_img_dict

Group Data by Scenario and Type

Define a function to group the data by scenario and type.
This organizes the data into a structure suitable for training.


In [ ]:
import numpy as np 
from collections import defaultdict 


def group_by_scenario_and_type(scenario_data, target_shape_s1, target_shape_s2):
    grouped_data = defaultdict(lambda: {'S1': [], 'S2': [], 'S1_filenames': [], 'S2_filenames': [], 'labels': [], 'dates': []})
    
    for scenario, data in scenario_data.items():
        for img_dict, lbl, fname, date in zip(data['images'], data['labels'], data['filenames'], data['dates']):
            if 'S1' in fname:
                resized_img_dict = resize_and_convert_images(img_dict, target_shape_s1)
                grouped_data[scenario]['S1'].append(resized_img_dict)
                grouped_data[scenario]['S1_filenames'].append(fname)
            elif 'S2' in fname:
                resized_img_dict = resize_and_convert_images(img_dict, target_shape_s2)
                grouped_data[scenario]['S2'].append(resized_img_dict)
                grouped_data[scenario]['S2_filenames'].append(fname)
            grouped_data[scenario]['labels'].append(lbl)
            grouped_data[scenario]['dates'].append(date)
            print(f"Scenario: {scenario}, File: {fname}, Type: {'S1' if 'S1' in fname else 'S2'}, Date: {date}, Label: {lbl}")
    
    return grouped_data

Group Training, Validation, and Testing Data

Group the training, validation, and testing data by scenario and type.


In [ ]:
target_shape_s1 = (522, 544, 1)
target_shape_s2 = (522, 544, 3)

train_grouped = group_by_scenario_and_type(train_data, target_shape_s1, target_shape_s2)
test_grouped = group_by_scenario_and_type(test_data, target_shape_s1, target_shape_s2)
validation_grouped = group_by_scenario_and_type(val_data, target_shape_s1, target_shape_s2)

Inspect Training Data Structure

Define a function to inspect the structure of the training data.
This helps in understanding the shape and type of each key in the data

In [ ]:
def inspect_train_grouped_structure(train_grouped):
    for scenario_id, data in train_grouped.items():
        print(f"Scenario ID: {scenario_id}")
        for key, value in data.items():
            if isinstance(value, list) and len(value) > 0:
                item = value[0]
                if isinstance(item, dict):
                    print(f"  Key: {key}, Type: List of Dicts")
                    for sub_key in item:
                        print(f"    - Dict Key: {sub_key}, Shape/Type: {np.shape(item[sub_key])}")
                else:
                    print(f"  Key: {key}, Type: List of {type(item).__name__}")
            else:
                print(f"  Key: {key}, Type: {type(value).__name__}, Shape/Type: {np.shape(value)}")
        print()
        
# Assuming 'train_grouped' is defined
inspect_train_grouped_structure(train_grouped)

In [ ]:
def resize_image_to_shape(img, target_shape):
    """
    Resizes an image to the target shape.
    """
    if img.shape != target_shape:
        img = resize(img, target_shape, preserve_range=True, anti_aliasing=True)
    return img


Prepare Data for Model

Define a function to prepare the data for the model.
This function handles resizing and stacking the images for each scenario.


In [ ]:
def prepare_data_for_model(scenario_data, target_shape_s1, target_shape_s2):
    sequences = []

    for scenario_id, data in scenario_data.items():
        s1_images, s2_images, s1_filenames, s2_filenames, labels, dates = [], [], [], [], [], []

        # Gather all images and metadata into a unified list
        all_data = []

        # Process S1 images
        for img_dict, fname, lbl, dt in zip(data['S1'], data['S1_filenames'], data['labels'], data['dates']):
            if 'VH' in img_dict:
                all_data.append({'image': img_dict['VH'], 'type': 'S1', 'filename': fname, 'label': lbl, 'date': dt})
            else:
                print(f"Skipping S1 image {fname} as it does not contain a 'VH' key.")

        # Process S2 images
        for img_dict, fname, lbl, dt in zip(data['S2'], data['S2_filenames'], data['labels'], data['dates']):
            if 'rgb' in img_dict:
                all_data.append({'image': img_dict['rgb'], 'type': 'S2', 'filename': fname, 'label': lbl, 'date': dt})
            else:
                print(f"Skipping S2 image {fname} as it does not contain an 'rgb' key.")

        # Sort the unified list by date
        all_data.sort(key=lambda x: x['date'])

        # Separate the sorted data
        for entry in all_data:
            if entry['type'] == 'S1':
                s1_images.append(resize_image_to_shape(entry['image'], target_shape_s1))
                s1_filenames.append(entry['filename'])
            else:  # entry['type'] == 'S2'
                s2_images.append(resize_image_to_shape(entry['image'], target_shape_s2))
                s2_filenames.append(entry['filename'])
            labels.append(entry['label'])
            dates.append(entry['date'])

        # Stack images to form sequences
        s1_images_np = np.stack(s1_images, axis=0) if s1_images else np.empty((0,) + target_shape_s1)
        s2_images_np = np.stack(s2_images, axis=0) if s2_images else np.empty((0,) + target_shape_s2)

        sequence_data = {
            's1_images': s1_images_np,
            's2_images': s2_images_np,
            's1_filenames': s1_filenames,
            's2_filenames': s2_filenames,
            'labels': np.array(labels),
            'dates': np.array(dates),
            'scenario': scenario_id
        }

        sequences.append(sequence_data)
        print(f"Finished processing scenario {scenario_id} with {s1_images_np.shape[0]} S1 images and {s2_images_np.shape[0]} S2 images")
        print(f"S1 Filenames: {s1_filenames}")
        print(f"S2 Filenames: {s2_filenames}")

    return sequences

In [ ]:
train_sequences = prepare_data_for_model(train_grouped, target_shape_s1, target_shape_s2)
val_sequences = prepare_data_for_model(validation_grouped, target_shape_s1, target_shape_s2)
test_sequences = prepare_data_for_model(test_grouped, target_shape_s1, target_shape_s2)

Build and Compile the Model

Define the neural network architecture using Keras layers.
The model uses ConvLSTM layers for both Sentinel-1 and Sentinel-2 inputs.


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Masking, ConvLSTM2D, BatchNormalization, Dense, Flatten, concatenate, GlobalAveragePooling2D, Embedding, TimeDistributed, GlobalAveragePooling3D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K
import tensorflow as tf
from keras.layers import Reshape
from tensorflow.keras.regularizers import l2
from sklearn.utils import class_weight
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

# Define input shapes
s1_input_shape = (None, 522, 544, 1)
s2_input_shape = (None, 522, 544, 3)
s1_input = Input(shape=s1_input_shape, name='S1_input')
s2_input = Input(shape=s2_input_shape, name='S2_input')

# S1 images processing branch
s1_masked = Masking(mask_value=0.0)(s1_input)
s1_branch = ConvLSTM2D(2, (3, 3), activation='relu', return_sequences=True, kernel_regularizer=l2(0.01), padding="same")(s1_masked)
s1_branch = BatchNormalization()(s1_branch)
s1_branch = Dropout(0.5)(s1_branch)
s1_branch = ConvLSTM2D(2, (3, 3), activation='relu', return_sequences=True, kernel_regularizer=l2(0.01), padding="same")(s1_branch)
s1_branch = Dropout(0.5)(s1_branch)

# S2 images processing branch
s2_masked = Masking(mask_value=0.0)(s2_input)
s2_branch = ConvLSTM2D(2, (3, 3), activation='relu', return_sequences=True, kernel_regularizer=l2(0.01), padding="same")(s2_masked)
s2_branch = BatchNormalization()(s2_branch)
s2_branch = Dropout(0.5)(s2_branch)
s2_branch = ConvLSTM2D(2, (3, 3), activation='relu', return_sequences=True, kernel_regularizer=l2(0.01), padding="same")(s2_branch)
s2_branch = Dropout(0.5)(s2_branch)

s1_branch = Reshape(target_shape=(-1, s1_branch.shape[2] * s1_branch.shape[3] * s1_branch.shape[4]))(s1_branch)
s2_branch = Reshape(target_shape=(-1, s2_branch.shape[2] * s2_branch.shape[3] * s2_branch.shape[4]))(s2_branch)

# Combine outputs from both branches
combined = concatenate([s1_branch, s2_branch], axis=-1)

# Apply TimeDistributed to process each timestep
final_layer = TimeDistributed(Dense(2, activation='relu'))(combined)
output = TimeDistributed(Dense(1, activation='sigmoid'))(final_layer)

learning_rate = 7e-6
optimizer = SGD(learning_rate=learning_rate, clipnorm=1.0)

# Create and compile the model
model = Model(inputs=[s1_input, s2_input], outputs=output)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

TensorFlow and Keras Libraries
Load necessary libraries for deep learning and data processing.



Padding Image Sequences
Define a function to pad image sequences to the same maximum length using numpy.


Padding Label Sequences
Define a function to pad or truncate label sequences to match the given target length.

Custom Weighted Binary Crossentropy
Define a custom weighted binary crossentropy loss function.

Calculate Class Weights
Define a function to calculate class weights for handling imbalanced data.

### Train the Model
Loop through the training epochs and train the model on the training data, while validating on the validation data.

In [ ]:
# Define a function to pad image sequences
def pad_image_sequences(sequences, maxlen=None, target_shape=None):
    """Pad image sequences to the same maximum length using numpy."""
    padded_sequences = []
    for seq in sequences:
        padded_seq = []
        for img in seq:
            # Padding the image to match the target shape
            padded_img = np.pad(img,
                                pad_width=((0, target_shape[0] - img.shape[0]), 
                                           (0, target_shape[1] - img.shape[1]), 
                                           (0, target_shape[2] - img.shape[2])),
                                mode='constant',
                                constant_values=0)
            padded_seq.append(padded_img)
        # Pad or truncate the sequence to maxlen
        if len(padded_seq) < maxlen:
            padded_seq.extend([np.zeros(target_shape) for _ in range(maxlen - len(padded_seq))])
        else:
            padded_seq = padded_seq[:maxlen]
        padded_sequences.append(padded_seq)
    result = np.array(padded_sequences)
    print(f"pad_image_sequences result shape: {result.shape}")
    return result

# Define a function to pad label sequences
def pad_labels_to_match_images(labels, target_length):
    """Pad or truncate label sequences to match the given target length."""
    adjusted_labels = []
    for lbl in labels:
        adjusted_lbl = lbl[:target_length] if len(lbl) > target_length else np.pad(lbl, (0, target_length - len(lbl)), 'constant')
        adjusted_labels.append(adjusted_lbl)
    result = np.array(adjusted_labels)
    print(f"pad_labels_to_match_images result shape: {result.shape}")
    return result

# Define a function for weighted binary crossentropy loss
def weighted_binary_crossentropy(weights):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        return tf.reduce_mean(-weights[1] * y_true * tf.math.log(y_pred + 1e-7) -
                              weights[0] * (1 - y_true) * tf.math.log(1 - y_pred + 1e-7))
    return loss

# Define a function to calculate class weights
def calculate_class_weights(y_train):
    flat_y = np.concatenate(y_train).ravel()
    weights = class_weight.compute_class_weight('balanced', classes=np.unique(flat_y), y=flat_y)
    return {0: weights[0], 1: weights[1]}

# Initialize training variables
num_epochs = 10
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

target_shape_s1 = (522, 544, 1)
target_shape_s2 = (522, 544, 3)

# Prepare training data
X_train_s1 = [seq['s1_images'] for seq in train_sequences]
X_train_s2 = [seq['s2_images'] for seq in train_sequences]
y_train = [seq['labels'] for seq in train_sequences]

# Determine maximum sequence length
max_len_train = max(max([len(seq) for seq in X_train_s1]),
                    max([len(seq) for seq in X_train_s2]))

# Pad S1 and S2 training data
X_train_s1_padded = [pad_image_sequences([img], maxlen=max_len_train, target_shape=target_shape_s1) for img in X_train_s1]
X_train_s2_padded = [pad_image_sequences([img], maxlen=max_len_train, target_shape=target_shape_s2) for img in X_train_s2]
y_train_padded = [pad_labels_to_match_images([lbl], max_len_train) for lbl in y_train]

# Prepare validation data
X_val_s1 = [seq['s1_images'] for seq in val_sequences]
X_val_s2 = [seq['s2_images'] for seq in val_sequences]
y_val = [seq['labels'] for seq in val_sequences]

# Determine maximum sequence length for validation data
max_len_val = max(max([len(seq) for seq in X_val_s1]),
                  max([len(seq) for seq in X_val_s2]))

# Pad S1 and S2 validation data
X_val_s1_padded = [pad_image_sequences([img], maxlen=max_len_val, target_shape=target_shape_s1) for img in X_val_s1]
X_val_s2_padded = [pad_image_sequences([img], maxlen=max_len_val, target_shape=target_shape_s2) for img in X_val_s2]
y_val_padded = [pad_labels_to_match_images([lbl], max_len_val) for lbl in y_val]

# Calculate class weights based on true labels
weights = calculate_class_weights(y_train_padded)
print("Class weights:", weights)

# Compile the model with the weighted loss
model.compile(optimizer='adam', loss=weighted_binary_crossentropy(weights), metrics=['accuracy'])

# Train the model
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    epoch_loss = []
    epoch_accuracy = []
    val_epoch_loss = []
    val_epoch_accuracy = []

    # Training loop
    for s1_batch, s2_batch, labels_batch in zip(X_train_s1_padded, X_train_s2_padded, y_train_padded):
        history = model.train_on_batch([s1_batch, s2_batch], labels_batch)
        epoch_loss.append(history[0])
        epoch_accuracy.append(history[1])

    avg_loss = np.mean(epoch_loss)
    avg_accuracy = np.mean(epoch_accuracy)
    train_losses.append(avg_loss)
    train_accuracies.append(avg_accuracy)
    print(f"Training - Epoch {epoch+1} - Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.4f}")

    # Validation loop
    for s1_batch, s2_batch, labels_batch in zip(X_val_s1_padded, X_val_s2_padded, y_val_padded):
        val_history = model.test_on_batch([s1_batch, s2_batch], labels_batch)
        val_epoch_loss.append(val_history[0])
        val_epoch_accuracy.append(val_history[1])

    val_avg_loss = np.mean(val_epoch_loss)
    val_avg_accuracy = np.mean(val_epoch_accuracy)
    val_losses.append(val_avg_loss)
    val_accuracies.append(val_avg_accuracy)
    print(f"Validation - Epoch {epoch+1} - Loss: {val_avg_loss:.4f}, Accuracy: {val_avg_accuracy:.4f}")

    # Adjust weights after the first epoch
    if epoch == 0:
        weights = calculate_class_weights(y_train_padded)
        print("Adjusted weights after first epoch:", weights)
        model.compile(optimizer='adam', loss=weighted_binary_crossentropy(weights), metrics=['accuracy'])

# Save the trained model
model.save("trainedfinal_model.h5")

# Plot training and validation metrics
epochs = list(range(1, num_epochs + 1))

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over epochs')
plt.legend()

plt.tight_layout()
plt.savefig("training_validation_plot_final.png")
plt.show()

Same preprocess step to fix dimension as did it with train and validation data

In [ ]:
# Prepare test data
X_test_s1 = [seq['s1_images'] for seq in test_sequences]
X_test_s2 = [seq['s2_images'] for seq in test_sequences]
y_test = [seq['labels'] for seq in test_sequences]

max_len_test = max(max([len(seq) for seq in X_test_s1]), max([len(seq) for seq in X_test_s2]))

target_shape_s1 = (522, 544, 1)
target_shape_s2 = (522, 544, 3)

# Pad test data
X_test_s1_padded = [pad_image_sequences([img], maxlen=max_len_test, target_shape=target_shape_s1) for img in X_test_s1]
X_test_s2_padded = [pad_image_sequences([img], maxlen=max_len_test, target_shape=target_shape_s2) for img in X_test_s2]
y_test_padded = [pad_labels_to_match_images([lbl], max_len_test) for lbl in y_test]

### Making Predictions
Make predictions on the test data using the trained model. y_test results are padded to the length of S1 to match dimensionsionality

In [ ]:
predictions = []
for s1_batch, s2_batch in zip(X_test_s1, X_test_s2):
    # Determine the maximum length
    max_len = max(len(s1_batch), len(s2_batch))
    
    # Pad s1_batch and s2_batch to the maximum length
    s1_batch_padded = pad_image_sequences([s1_batch], maxlen=max_len, target_shape=(522, 544, 1))
    s2_batch_padded = pad_image_sequences([s2_batch], maxlen=max_len, target_shape=(522, 544, 3))
    
    # Predict
    prediction = model.predict([s1_batch_padded, s2_batch_padded])
    predictions.append(prediction)

predicted_labels = [(pred > 0.5).astype(int).flatten() for pred in predictions]


In [ ]:
print(predicted_labels)

### Calculate Metrics
Calculate accuracy, precision, recall, and F1 score for each test scenario.

### Average Metrics
Calculate and print the average metrics across all test scenarios.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# Calculate metrics for each scenario
# Calculate metrics for each scenario
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for i in range(len(y_test_padded)):
    y_true = y_test_padded[i].flatten()
    y_pred = predicted_labels[i]
    
    # Align lengths
    max_len = max(len(y_true), len(y_pred))
    if len(y_true) < max_len:
        y_true = np.pad(y_true, (0, max_len - len(y_true)), 'constant')
    if len(y_pred) < max_len:
        y_pred = np.pad(y_pred, (0, max_len - len(y_pred)), 'constant')
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Print the metrics for each scenario
    print(f"Scenario {i + 1}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print()

# Calculate average metrics across scenarios
avg_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_f1 = sum(f1_scores) / len(f1_scores)

# Print average metrics
print("Average Metrics:")
print(f"Average Accuracy: {avg_accuracy:.4f}")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1 Score: {avg_f1:.4f}")